In [1]:
import numpy as np
from src.data.bridge_site import get_dataloaders

In [2]:
data_order_large = ['population', 'osm_img', 'elevation', 'slope', 'roads', 'waterways', 'admin_bounds_qgis']
data_order_small = ['osm_img', 'slope', 'waterways', 'admin_bounds_qgis']

In [3]:
dataloaders = get_dataloaders(batch_size=64, tile_size=300, data_version="v1", data_order=data_order_large, 
                              transform=False, test_batch_size=64, use_augment=False, 
                              use_rnd_center_point=False, use_several_test_samples=False)

train sampler: 1630 samples in total (815 pos, 815 neg) ([Rwanda] 815 pos 815 neg [Uganda] 0 pos 0 neg)
val sampler: 484 samples in total (242 pos, 242 neg) ([Rwanda] 242 pos 242 neg [Uganda] 0 pos 0 neg)
test sampler: 740 samples in total (370 pos, 370 neg) ([Rwanda] 370 pos 370 neg [Uganda] 0 pos 0 neg)
test sampler: 500 samples in total (250 pos, 250 neg) ([Rwanda] 0 pos 0 neg [Uganda] 250 pos 250 neg)


In [11]:
sorted(list(dataloaders[0].sampler))

[0,
 2,
 26,
 27,
 28,
 29,
 30,
 32,
 33,
 34,
 35,
 37,
 40,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 54,
 59,
 60,
 61,
 62,
 63,
 64,
 66,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 79,
 81,
 83,
 90,
 91,
 93,
 98,
 99,
 100,
 101,
 102,
 105,
 106,
 108,
 110,
 111,
 112,
 113,
 115,
 118,
 119,
 120,
 121,
 122,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 145,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 166,
 170,
 172,
 173,
 178,
 179,
 180,
 181,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 202,
 203,
 205,
 206,
 207,
 210,
 212,
 214,
 215,
 216,
 217,
 222,
 227,
 228,
 239,
 240,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288

In [12]:
for data_order, data_order_name in [[data_order_small, "small"], [data_order_large, "large"]]:
    for version in ["v1", "v2"]:
        for tile_size in [300, 600, 1200]:
            numpy_save = {}
            dataloaders = get_dataloaders(
                batch_size=64, tile_size=tile_size, data_version=version, data_order=data_order, 
                transform=False, test_batch_size=64, use_augment=False,
                use_rnd_center_point=False, use_several_test_samples=False)
            for i, name in [[0, "train"], [1, "val"], [2, "test_rwanda"], [3, "test_uganda"]]:
                X = []
                Y = []
                dataset = dataloaders[i].dataset
                indices = list(dataloaders[0].sampler)
                for j in indices:
                    if dataset.train_gdf.iloc[j].pos_neg not in ["pos", "neg"]:
                        continue
                    x, y = dataset[j]
                    num_channels = x.shape[0]
                    X.append(x.view(1, num_channels, -1).numpy())
                    Y.append(y)
                X = np.concatenate(X, 0)
                Y = np.array(Y)
                numpy_save["{}_X".format(name)] = X
                numpy_save["{}_Y".format(name)] = Y
            np.savez(
                "data_{}_{}_{}.npz".format(data_order_name, tile_size, version),
                **numpy_save)

train sampler: 1630 samples in total (815 pos, 815 neg) ([Rwanda] 815 pos 815 neg [Uganda] 0 pos 0 neg)
val sampler: 484 samples in total (242 pos, 242 neg) ([Rwanda] 242 pos 242 neg [Uganda] 0 pos 0 neg)
test sampler: 740 samples in total (370 pos, 370 neg) ([Rwanda] 370 pos 370 neg [Uganda] 0 pos 0 neg)
test sampler: 500 samples in total (250 pos, 250 neg) ([Rwanda] 0 pos 0 neg [Uganda] 250 pos 250 neg)
train sampler: 1630 samples in total (815 pos, 815 neg) ([Rwanda] 815 pos 815 neg [Uganda] 0 pos 0 neg)
val sampler: 484 samples in total (242 pos, 242 neg) ([Rwanda] 242 pos 242 neg [Uganda] 0 pos 0 neg)
test sampler: 740 samples in total (370 pos, 370 neg) ([Rwanda] 370 pos 370 neg [Uganda] 0 pos 0 neg)
test sampler: 500 samples in total (250 pos, 250 neg) ([Rwanda] 0 pos 0 neg [Uganda] 250 pos 250 neg)
train sampler: 1630 samples in total (815 pos, 815 neg) ([Rwanda] 815 pos 815 neg [Uganda] 0 pos 0 neg)
val sampler: 484 samples in total (242 pos, 242 neg) ([Rwanda] 242 pos 242 neg

In [13]:
import numpy

data = np.load("data_large_1200_v1.npz")
print(data["train_X"].shape, data["train_Y"].shape)
print(data["val_X"].shape, data["val_Y"].shape)
print(data["test_rwanda_X"].shape, data["test_rwanda_Y"].shape)
print(data["test_uganda_X"].shape, data["test_uganda_Y"].shape)

print()

data = np.load("data_small_600_v1.npz")
print(data["train_X"].shape, data["train_Y"].shape)
print(data["val_X"].shape, data["val_Y"].shape)
print(data["test_rwanda_X"].shape, data["test_rwanda_Y"].shape)
print(data["test_uganda_X"].shape, data["test_uganda_Y"].shape)

(1630, 9, 2304) (1630,)
(1630, 9, 2304) (1630,)
(1630, 9, 2304) (1630,)
(1630, 9, 2304) (1630,)

(1630, 6, 576) (1630,)
(1630, 6, 576) (1630,)
(1630, 6, 576) (1630,)
(1630, 6, 576) (1630,)
